# TensorFlow

**[MDT-01]** 必要なモジュールをインポートして、乱数のシードを設定します。

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20160703)
tf.set_random_seed(20160703)

/Users/yohei/.pyenv/versions/anaconda3-5.0.0/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


**[MDT-02]** MNISTのデータセットを用意します。

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


**[MDT-03]** 畳込みフィルターが1層のCNNを表現するクラスを定義します。

In [6]:
class SingleCNN:
    def __init__(self, num_filters, num_units):
        # tf.Graph().as_default()のコンテキストマネージャーにより異なるグラフを(with構文内では)default Graphとして扱うことができる．
        # また複数のグラフを作成することができる．
        with tf.Graph().as_default():
            self.prepare_model(num_filters, num_units)
            self.prepare_session()

    def prepare_model(self, num_filters, num_units):
        num_units1 = 14*14*num_filters
        num_units2 = num_units
        
        with tf.name_scope('input'):
            x = tf.placeholder(tf.float32, [None, 784], name='input')
            x_image = tf.reshape(x, [-1,28,28,1])

        with tf.name_scope('convolution'):
            W_conv = tf.Variable(
                tf.truncated_normal([5,5,1,num_filters], stddev=0.1),
                name='conv-filter')
            h_conv = tf.nn.conv2d(
                x_image, W_conv, strides=[1,1,1,1], padding='SAME',
                name='filter-output')

        with tf.name_scope('pooling'):            
            h_pool =tf.nn.max_pool(h_conv, ksize=[1,2,2,1],
                                   strides=[1,2,2,1], padding='SAME',
                                   name='max-pool')
            h_pool_flat = tf.reshape(h_pool, [-1, 14*14*num_filters],
                                     name='pool-output')

        with tf.name_scope('fully-connected'):
            w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
            b2 = tf.Variable(tf.zeros([num_units2]))
            hidden2 = tf.nn.relu(tf.matmul(h_pool_flat, w2) + b2,
                                 name='fc-output')

        with tf.name_scope('softmax'):
            w0 = tf.Variable(tf.zeros([num_units2, 10]))
            b0 = tf.Variable(tf.zeros([10]))
            p = tf.nn.softmax(tf.matmul(hidden2, w0) + b0,
                              name='softmax-output')
            
        with tf.name_scope('optimizer'):
            t = tf.placeholder(tf.float32, [None, 10], name='labels')
            loss = -tf.reduce_sum(t * tf.log(p), name='loss')
            train_step = tf.train.AdamOptimizer(0.0005).minimize(loss)
            
        with tf.name_scope('evaluator'):
            correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction,
                                              tf.float32), name='accuracy')
            
        # 監視したいTensorを追加
        tf.summary.scalar("loss", loss)
        tf.summary.scalar("accuracy", accuracy)
        tf.summary.histogram("convolution_filters", W_conv)
        
        # 外から使うであろう変数をインスタンス変数として登録．
        self.x, self.t, self.p = x, t, p
        self.train_step = train_step
        self.loss = loss
        self.accuracy = accuracy
        
    def prepare_session(self):
        sess = tf.Session()  # graph引数に指定がない場合，default_graphが指定される
        sess.run(tf.global_variables_initializer())
        summary = tf.summary.merge_all()  # summary を全てマージする． 登録した変数を1まとめにする．
        # FileWriterクラスを作成， どこに記述するかのインスタンスを生成
        writer = tf.summary.FileWriter("/tmp/mnist_df_logs", graph=sess.graph)  # graph引数でどのgraphを書き出すのかを指定．  
        
        self.sess = sess
        self.summary = summary
        self.writer = writer

**[MDT-04]** TensorBoard用のデータ出力ディレクトリーを削除して初期化しておきます。

In [7]:
!rm -rf /tmp/mnist_df_logs

**[MDT-05]** パラメーターの最適化を4000回繰り返します。テストセットに対して約98%の正解率が得られます。

In [8]:
cnn = SingleCNN(16, 1024)

i = 0
for _ in range(4000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(100)
    cnn.sess.run(cnn.train_step, feed_dict={cnn.x:batch_xs, cnn.t:batch_ts})
    if i % 50 == 0:
        summary, loss_val, acc_val = cnn.sess.run(
            [cnn.summary, cnn.loss, cnn.accuracy],
            feed_dict={cnn.x:mnist.test.images, cnn.t:mnist.test.labels}) # テストデータの結果(loss, accuracy, convolution_filters)をサマリーに記述
        print ('Step: %d, Loss: %f, Accuracy: %f'
               % (i, loss_val, acc_val))
        # FileWriterインスタンス(writer)に書き出すことで TensorBoadに値を記録する． 二個目の引数は最適化処理の実施回数を与えている．
        # global_step : Number. Optional global step value to record with the summary.
        cnn.writer.add_summary(summary, i)

Step: 50, Loss: 2899.623047, Accuracy: 0.915400
Step: 100, Loss: 2314.003906, Accuracy: 0.932500
Step: 150, Loss: 1901.670410, Accuracy: 0.946400
Step: 200, Loss: 2054.363525, Accuracy: 0.935400
Step: 250, Loss: 1480.421143, Accuracy: 0.954800
Step: 300, Loss: 1418.499023, Accuracy: 0.956100
Step: 350, Loss: 1334.980713, Accuracy: 0.958200
Step: 400, Loss: 1244.540283, Accuracy: 0.961600
Step: 450, Loss: 1109.972534, Accuracy: 0.965000
Step: 500, Loss: 1126.324951, Accuracy: 0.965700
Step: 550, Loss: 1109.605957, Accuracy: 0.966200
Step: 600, Loss: 1049.442017, Accuracy: 0.966700
Step: 650, Loss: 927.259644, Accuracy: 0.971200
Step: 700, Loss: 945.089539, Accuracy: 0.972300
Step: 750, Loss: 961.402710, Accuracy: 0.968600
Step: 800, Loss: 867.125061, Accuracy: 0.972200
Step: 850, Loss: 839.944946, Accuracy: 0.974900
Step: 900, Loss: 866.591675, Accuracy: 0.972400
Step: 950, Loss: 809.067505, Accuracy: 0.975900
Step: 1000, Loss: 824.606628, Accuracy: 0.975300
Step: 1050, Loss: 947.164062

# Keras

In [1]:
from keras.models import Sequential
from keras.layers.core import Activation, Lambda, Dense, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from keras.callbacks import TensorBoard, EarlyStopping
from keras.initializers import TruncatedNormal, Zeros

/Users/yohei/.pyenv/versions/anaconda3-5.0.0/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
np.random.seed(20160703)

In [5]:
model = Sequential()

In [6]:
num_filters, num_units =16, 1024
num_units1 = 14*14*num_filters
num_units2 = num_units

with K.name_scope('CNN'):
    model.add(Conv2D(filters=num_filters, kernel_size=5,
                     padding="same",
                     data_format="channels_last",
                     use_bias=False,
                     input_shape=(28, 28, 1),
                    kernel_initializer=TruncatedNormal(stddev=0.1),
                    name='convolution'
                    ))

    with K.name_scope('pooling'): # TensorFlowのname_scopeが使える． 色々とTensorFlowと変換性がある
        model.add(MaxPooling2D(pool_size=(2, 2),
                           padding="same", 
                           data_format="channels_last"))
        model.add(Flatten())  #  入力を平滑化する．バッチサイズに影響されない．

    model.add(Dense(units=num_units2, activation='relu', name='fully-connected'))
    model.add(Dense(units=10, activation='softmax', name='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
convolution (Conv2D)         (None, 28, 28, 16)        400       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
fully-connected (Dense)      (None, 1024)              3212288   
_________________________________________________________________
softmax (Dense)              (None, 10)                10250     
Total params: 3,222,938
Trainable params: 3,222,938
Non-trainable params: 0
_________________________________________________________________


In [9]:
!rm -rf /tmp/mnist_df_logs

In [10]:
tensorboad = TensorBoard(log_dir='/tmp/mnist_df_logs', histogram_freq=1)  # histogram_freq=50で 50 epochごとに記録することになる
earlystopping = EarlyStopping(monitor='val_loss',  patience=5, verbose=1)

In [11]:
model.compile(loss='categorical_crossentropy',
             optimizer=Adam(lr=0.0005),
             metrics=['accuracy'])

In [12]:
x_train, t_train = mnist.train.images,mnist.train.labels
x_train.shape, t_train.shape

((55000, 784), (55000, 10))

In [13]:
x_test, t_test = mnist.test.images,mnist.test.labels
x_test.shape, t_test.shape

((10000, 784), (10000, 10))

In [14]:
# 学習
x_train = x_train.reshape([-1,28,28,1])
x_test = x_test.reshape([-1,28,28,1])

hist = model.fit(x_train, t_train, batch_size=100,
                         epochs=10,
                         validation_data=(x_test, t_test),
                         callbacks=[tensorboad, earlystopping])

Train on 55000 samples, validate on 10000 samples
Epoch 1/10
55000/55000 [==============================] - 114s 2ms/step - loss: 0.2113 - acc: 0.9391 - val_loss: 0.0960 - val_acc: 0.9706
Epoch 2/10
55000/55000 [==============================] - 113s 2ms/step - loss: 0.0674 - acc: 0.9798 - val_loss: 0.0610 - val_acc: 0.9812
Epoch 3/10
55000/55000 [==============================] - 114s 2ms/step - loss: 0.0386 - acc: 0.9879 - val_loss: 0.0574 - val_acc: 0.9807
Epoch 4/10
55000/55000 [==============================] - 115s 2ms/step - loss: 0.0260 - acc: 0.9918 - val_loss: 0.0514 - val_acc: 0.9845
Epoch 5/10
55000/55000 [==============================] - 114s 2ms/step - loss: 0.0163 - acc: 0.9952 - val_loss: 0.0495 - val_acc: 0.9840
Epoch 6/10
55000/55000 [==============================] - 115s 2ms/step - loss: 0.0110 - acc: 0.9968 - val_loss: 0.0475 - val_acc: 0.9866
Epoch 7/10
55000/55000 [==============================] - 116s 2ms/step - loss: 0.0087 - acc: 0.9975 - val_loss: 0.0691 - 

In [15]:
model.input

<tf.Tensor 'convolution_input:0' shape=(?, 28, 28, 1) dtype=float32>

In [16]:
model.output

<tf.Tensor 'softmax/Softmax:0' shape=(?, 10) dtype=float32>

以上のように裏ではTensorが動いている

In [18]:
model.layers